In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1" 

In [2]:
from datasets import load_dataset

# dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train[:100]", trust_remote_code=True)
metadata_file = "/Users/allen/Documents/Wav2vec_Project/asr/common_voice/cv-valid-train-downsampled.csv"
dataset = load_dataset("audiofolder", data_dir="/Users/allen/Documents/Wav2vec_Project/asr/common_voice/wav-files/", drop_metadata=False)

/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd 
metadata= pd.read_csv(metadata_file)
metadata['filename'] = metadata['filename'].str.replace(
    'cv-valid-train/', ''
).str.replace(
    '.mp3', '.wav'
)

In [4]:
print (metadata)

              filename                                               text  \
0    sample-182715.wav  the fine manufacturing company was a bookkeepe...   
1    sample-114956.wav  the mixture took on a reddish color almost the...   
2    sample-139880.wav                                enough said the boy   
3    sample-034538.wav  then he sat in the sunfilled doorway smoking t...   
4    sample-142907.wav  i've just guaranteed the bank sufficient funds...   
..                 ...                                                ...   
191  sample-042142.wav                                    you have no car   
192  sample-110037.wav  they looked for the fallen mass but found nothing   
193  sample-101888.wav     his way isn't the same as mine nor mine as his   
194  sample-135316.wav                             he wasn't an alchemist   
195  sample-034446.wav                              now look what you did   

     up_votes  down_votes       age gender   accent  duration  
0          

In [5]:
def match_transcription(example):
    # Extract just the filename from the audio path
    audio_filename = example["audio"]["path"].split("/")[-1]
    
    # Find the matching transcription in the metadata
    text_row = metadata[metadata["filename"] == audio_filename]
    print (audio_filename,text_row)
    # Return the transcription if found, else return None
    return {"text": text_row["text"].iloc[0] if not text_row.empty else None,"filepath": example["audio"]["path"] if not text_row.empty else None}
dataset = dataset.map(match_transcription)

In [ ]:
# print (dataset['train']['audio'])

[{'path': None, 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        0.00000000e+00, -3.05175781e-05,  0.00000000e+00]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.05175781e-05,  0.00000000e+00,  0.00000000e+00]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.05175781e-05, -3.05175781e-05, -3.05175781e-05]), 'sampling_rate': 16000}, {'path': None, 'array': array([0., 0., 0., ..., 

In [7]:
print(dataset.column_names)

{'train': ['audio', 'text', 'filepath']}


In [8]:
print(metadata["filename"].head())  # Print the first few filenames

0    sample-182715.wav
1    sample-114956.wav
2    sample-139880.wav
3    sample-034538.wav
4    sample-142907.wav
Name: filename, dtype: object


In [9]:
print (dataset['train']['text'][:5])

['the visual acuity is astonishing', 'how strange africa is thought the boy', 'until then he had considered the omens to be things of this world', 'i had seen all that it would presently bring me', 'my tax dollars pay for those public school proms']


In [10]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

def preprocess(batch):
    batch["input_values"] = batch["audio"]["array"]
    batch["input_length"] = len(batch["input_values"])
    return batch

dataset = dataset.map(preprocess, remove_columns=["audio"])

In [11]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import torch
def prepare_dataset(batch):
    # Process the audio input
    audio = processor.feature_extractor(
        batch["input_values"], 
        sampling_rate=16_000,
        return_attention_mask=False  # Explicitly set this argument for clarity
    )
    batch["input_values"] = audio.input_values[0]

    # Process the text target
    with processor.as_target_processor():
        text = processor.tokenizer(
            batch["text"], 
            return_attention_mask=False,  # Ensure this doesn't cause conflict
            truncation=True  # Prevent token overflow
        )
        batch["labels"] = text.input_ids

    return batch

dataset = dataset.map(prepare_dataset)

In [13]:
train_test_split = dataset['train'].train_test_split(test_size=0.3)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [14]:
from torch.nn.utils.rnn import pad_sequence

# Set the device to CPU
device = torch.device("cpu")

# Move the model and inputs to CPU
model.to(device)
# processor.to(device)

class CustomDataCollatorCTCWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        input_features = [torch.tensor(feature["input_values"]) for feature in features]
        label_features = [torch.tensor(feature["labels"]) for feature in features]

        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=self.processor.feature_extractor.padding_value)
        labels_padded = pad_sequence(label_features, batch_first=True, padding_value=-100)
        attention_masks = torch.zeros_like(input_features_padded).masked_fill(input_features_padded != self.processor.feature_extractor.padding_value, 1)

        return {
            "input_values": input_features_padded,
            "labels": labels_padded,
            "attention_mask": attention_masks
        }

data_collator = CustomDataCollatorCTCWithPadding(processor=processor)

In [15]:
import jiwer
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = jiwer.wer(label_str, pred_str)

    return {"wer": wer}

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_total_limit=2,
    bf16=True,  # Use bf16 instead of fp16
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    eval_steps=500,
    save_steps=500,
    push_to_hub=False,
    remove_unused_columns=False,
)

/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

/var/folders/p8/j65kmwm10d3d5lgjzjydyntm0000gn/T/ipykernel_41892/1212053796.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/51 [00:00<?, ?it/s]/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/torch/nn/functional.py:2687: UserWarning: The operator 'aten::_ctc_loss' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch.ctc_loss(
                                                  
 35%|███▌      | 18/51 [42:11<1:10:54, 128.93s/it]

{'eval_loss': 5898.0234375, 'eval_wer': 1.021015761821366, 'eval_runtime': 49.726, 'eval_samples_per_second': 1.187, 'eval_steps_per_second': 0.302, 'epoch': 1.0}


                                                  
 71%|███████   | 36/51 [44:16<01:00,  4.06s/it]

{'eval_loss': 5526.326171875, 'eval_wer': 1.021015761821366, 'eval_runtime': 4.8057, 'eval_samples_per_second': 12.277, 'eval_steps_per_second': 3.121, 'epoch': 2.0}


                                               
100%|██████████| 51/51 [45:44<00:00,  3.05s/it]

{'eval_loss': 5049.53662109375, 'eval_wer': 1.021015761821366, 'eval_runtime': 4.9566, 'eval_samples_per_second': 11.903, 'eval_steps_per_second': 3.026, 'epoch': 2.86}


100%|██████████| 51/51 [45:48<00:00, 53.88s/it]

{'train_runtime': 2748.0998, 'train_samples_per_second': 0.15, 'train_steps_per_second': 0.019, 'train_loss': 8511.229166666666, 'epoch': 2.86}


TrainOutput(global_step=51, training_loss=8511.229166666666, metrics={'train_runtime': 2748.0998, 'train_samples_per_second': 0.15, 'train_steps_per_second': 0.019, 'total_flos': 7.741184044157338e+16, 'train_loss': 8511.229166666666, 'epoch': 2.857142857142857})

In [18]:
metrics = trainer.evaluate()

100%|██████████| 15/15 [00:04<00:00,  3.52it/s]


In [19]:
print (metrics)

{'eval_loss': 5049.53662109375, 'eval_wer': 1.021015761821366, 'eval_runtime': 4.727, 'eval_samples_per_second': 12.482, 'eval_steps_per_second': 3.173, 'epoch': 2.857142857142857}


In [20]:
# Save the model and tokenizer
model.save_pretrained("./wav2vec2-large-960h-cv")


In [ ]:
## Manually Test the Model
# Load the model and processor (tokenizer) from the saved directory
test_model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-960h-cv")
test_processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-large-960h-cv")